# BiSeNetV3 + EfficientNetB1

In [31]:
# BiSeNettV3 + EfficientNetB1

import tensorflow as tf
from tensorflow.keras.layers import (Conv2D, BatchNormalization, ReLU,
                                     GlobalAveragePooling2D, Reshape, Multiply,
                                     Add, Input, Concatenate, Conv2DTranspose)
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB1

# Squeeze-and-Excitation Block (tidak berubah)
def se_block(x, reduction_ratio=16):
    filters = x.shape[-1]
    se = GlobalAveragePooling2D()(x)
    se = Reshape((1, 1, filters))(se)
    se = Conv2D(filters // reduction_ratio, 1, activation='relu')(se)
    se = Conv2D(filters, 1, activation='softmax')(se)
    return Multiply()([x, se])

# Adaptive Receptive Field Block
def adaptive_rf_block(x, filters):
    # Dilated convolutions for adaptive receptive fields
    dilate1 = Conv2D(filters, 3, padding='same', dilation_rate=1, activation='relu')(x)
    dilate2 = Conv2D(filters, 3, padding='same', dilation_rate=2, activation='relu')(x)
    dilate3 = Conv2D(filters, 3, padding='same', dilation_rate=4, activation='relu')(x)
    return Add()([dilate1, dilate2, dilate3])

# Detail Branch (sama seperti di BiSeNetV2)
def detail_branch(input_tensor):
    x = Conv2D(64, 3, 2, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(64, 3, 1, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Stage 2
    x = Conv2D(128, 3, 2, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    for _ in range(2):
        x_res = Conv2D(128, 3, 1, padding='same')(x)
        x_res = BatchNormalization()(x_res)
        x_res = ReLU()(x_res)
        x_res = Conv2D(128, 3, 1, padding='same')(x_res)
        x_res = BatchNormalization()(x_res)
        x = Add()([x, x_res])
        x = ReLU()(x)
    return x

# Semantic Branch with Context Embedding
def semantic_branch(input_tensor):
    # Use EfficientNetB1 as the backbone
    base_model = EfficientNetB1(weights='imagenet', include_top=False, input_tensor=input_tensor)

    # Extract features from a specific layer
    x = base_model.get_layer("block6a_expand_activation").output  # Layer EfficientNetB1
    x = Conv2D(128, 1, padding='same')(x)  # Channel reduction
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Context embedding
    x = adaptive_rf_block(x, filters=128)
    x = se_block(x)  # SE block for feature enhancement
    return x

# Feature Fusion Module (FFM)
def feature_fusion(detail, semantic):
    # Apply detail guidance
    detail_guidance = Conv2D(128, 3, padding='same')(detail)
    detail_guidance = BatchNormalization()(detail_guidance)
    detail_guidance = ReLU()(detail_guidance)

    # Upsample semantic branch output
    semantic = Conv2DTranspose(128, (4, 4), strides=(4, 4), padding='same')(semantic)
    semantic_guidance = Conv2D(128, 3, padding='same')(semantic)
    semantic_guidance = BatchNormalization()(semantic_guidance)
    semantic_guidance = ReLU()(semantic_guidance)

    # Concatenate and apply attention mechanism
    fusion = Concatenate()([detail_guidance, semantic_guidance])
    fusion = Conv2D(128, 3, padding='same')(fusion)
    fusion = BatchNormalization()(fusion)
    fusion = ReLU()(fusion)

    fusion = se_block(fusion)  # Adding SE block to refine features
    return fusion

# Create BiSeNetV3 Model with EfficientNetB1 backbone
def create_bisenetv3(input_shape=(256, 256, 3), num_classes=3):
    inputs = Input(shape=input_shape)

    # Detail Branch
    detail = detail_branch(inputs)

    # Semantic Branch
    semantic = semantic_branch(inputs)

    # Feature Fusion
    x = feature_fusion(detail, semantic)

    # Final layers with transpose convolution for upsampling
    x = Conv2D(128, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Upsample output to match input size
    x = Conv2DTranspose(num_classes, (4, 4), strides=(4, 4), padding='same', activation='softmax')(x)

    # Create model
    model = Model(inputs=inputs, outputs=x, name="BiSeNetV3_EfficientNetB1")
    return model

# Create the model
model = create_bisenetv3(input_shape=(256, 256, 3))
model.summary()

Model: "BiSeNetV3_EfficientNetB1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_51 (InputLayer)   │ (None, 128, 128, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_61 (Rescaling)      │ (None, 128, 128, 3)       │               0 │ input_layer_51[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ normalization_30              │ (None, 128, 128, 3)       │               7 │ rescaling_61[0][0]         │
│ (Normalization)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_62 (Rescaling)      │ (None, 128, 128, 3)       │               0 │ normalization_30[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv_pad (ZeroPadding2D) │ (None, 129, 129, 3)       │               0 │ rescaling_62[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv (Conv2D)            │ (None, 64, 64, 32)        │             864 │ stem_conv_pad[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_bn (BatchNormalization)  │ (None, 64, 64, 32)        │             128 │ stem_conv[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_activation (Activation)  │ (None, 64, 64, 32)        │               0 │ stem_bn[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_dwconv                │ (None, 64, 64, 32)        │             288 │ stem_activation[0][0]      │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_bn                    │ (None, 64, 64, 32)        │             128 │ block1a_dwconv[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_activation            │ (None, 64, 64, 32)        │               0 │ block1a_bn[0][0]           │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_squeeze            │ (None, 32)                │               0 │ block1a_activation[0][0]   │
│ (GlobalAveragePooling2D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reshape (Reshape)  │ (None, 1, 1, 32)          │               0 │ block1a_se_squeeze[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reduce (Conv2D)    │ (None, 1, 1, 8)           │             264 │ block1a_se_reshape[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_expand (Conv2D)    │ (None, 1, 1, 32)          │             28

 Total params: 3,554,750 (13.56 MB)

 Trainable params: 3,527,703 (13.46 MB)

 Non-trainable params: 27,047 (105.66 KB)

# Mobile U-Net + EfficientNetB1

In [2]:
# Mobile U-Net + EfficientNetB1
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetB1

def mobile_unet_efficientnet(input_size=(256, 256, 3), num_classes=3, alpha=1.0):
    """
    Mobile U-Net with EfficientNetB1 Backbone
    
    Parameters:
    - input_size: Input image dimensions
    - num_classes: Number of output classes
    - alpha: Width multiplier for depthwise separable convolutions
    """
    # Load EfficientNetB1 as encoder
    efficientnet_base = EfficientNetB1(
        include_top=False,
        weights='imagenet',
        input_shape=input_size
    )
    
    # Option to make backbone trainable for fine-tuning
    efficientnet_base.trainable = False
    
    # Encoder (Using EfficientNetB1 pretrained layers)
    skip1 = efficientnet_base.get_layer('block1a_activation').output  # 64x64
    skip2 = efficientnet_base.get_layer('block2a_activation').output  # 32x32
    skip3 = efficientnet_base.get_layer('block3a_activation').output  # 16x16
    skip4 = efficientnet_base.get_layer('block4a_activation').output  # 8x8
    
    # Bridge (deepest layer in U-Net)
    x = se_depthwise_conv_block(skip4, 1024, alpha, strides=(2, 2))  # 4x4x1024
    x = se_depthwise_conv_block(x, 1024, alpha)  # 4x4x1024
    
    # Decoder (Upsampling with Squeeze-and-Excitation)
    # Block 5
    x = layers.UpSampling2D(size=(2, 2))(x)  # 8x8x1024
    x = layers.concatenate([x, skip4])  # 8x8x1536
    x = se_depthwise_conv_block(x, 512, alpha)  # 8x8x512
    
    # Block 6
    x = layers.UpSampling2D(size=(2, 2))(x)  # 16x16x512
    x = layers.concatenate([x, skip3])  # 16x16x768
    x = se_depthwise_conv_block(x, 256, alpha)  # 16x16x256
    
    # Block 7
    x = layers.UpSampling2D(size=(2, 2))(x)  # 32x32x256
    x = layers.concatenate([x, skip2])  # 32x32x384
    x = se_depthwise_conv_block(x, 128, alpha)  # 32x32x128
    
    # Block 8
    x = layers.UpSampling2D(size=(2, 2))(x)  # 64x64x128
    x = layers.concatenate([x, skip1])  # 64x64x192
    x = se_depthwise_conv_block(x, 64, alpha)  # 64x64x64
    
    # Final upsampling and output with attention
    x = layers.UpSampling2D(size=(2, 2))(x)  # 128x128x64
    
    # Add CBAM (Convolutional Block Attention Module)
    x = cbam_block(x)
    
    # Final convolution
    outputs = layers.Conv2D(num_classes, 1, activation='softmax')(x)  # 128x128x3
    
    # Create the full model
    model = Model(
        inputs=efficientnet_base.input, 
        outputs=outputs, 
        name="256_MobileU-Net_EfficientNetB1"
    )
    
    return model

def se_depthwise_conv_block(inputs, pointwise_conv_filters, alpha, 
                             depth_multiplier=1, strides=(1, 1)):
    """
    Depthwise Separable Convolution Block with Squeeze-and-Excitation
    
    Parameters:
    - inputs: Input tensor
    - pointwise_conv_filters: Number of filters in pointwise conv
    - alpha: Width multiplier
    - depth_multiplier: Depth multiplier for depthwise conv
    - strides: Convolution strides
    """
    pointwise_conv_filters = int(pointwise_conv_filters * alpha)
    
    # Depthwise Convolution
    x = layers.DepthwiseConv2D(
        (3, 3), 
        padding='same', 
        depth_multiplier=depth_multiplier, 
        strides=strides, 
        use_bias=False
    )(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU(6.)(x)
    
    # Pointwise Convolution
    x = layers.Conv2D(
        pointwise_conv_filters, 
        (1, 1), 
        padding='same', 
        use_bias=False, 
        strides=(1, 1)
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU(6.)(x)
    
    # Squeeze-and-Excitation Block
    x = squeeze_excite_block(x)
    
    return x

def squeeze_excite_block(inputs, ratio=16):
    """
    Squeeze-and-Excitation Block for channel-wise attention
    
    Parameters:
    - inputs: Input tensor
    - ratio: Reduction ratio for channel compression
    """
    channels = inputs.shape[-1]
    
    # Squeeze
    x = layers.GlobalAveragePooling2D()(inputs)
    
    # Excitation
    x = layers.Dense(channels // ratio, activation='relu')(x)
    x = layers.Dense(channels, activation='softmax')(x)
    
    # Scale
    x = layers.Reshape((1, 1, channels))(x)
    return layers.Multiply()([inputs, x])

def cbam_block(inputs):
    """
    Convolutional Block Attention Module (CBAM)
    
    Parameters:
    - inputs: Input tensor
    """
    # Channel Attention
    channel_attention = channel_attention_block(inputs)
    channel_refined = layers.Multiply()([inputs, channel_attention])
    
    # Spatial Attention
    spatial_attention = spatial_attention_block(channel_refined)
    
    return layers.Multiply()([channel_refined, spatial_attention])

def channel_attention_block(inputs, ratio=16):
    """
    Channel Attention Module
    
    Parameters:
    - inputs: Input tensor
    - ratio: Reduction ratio
    """
    channels = inputs.shape[-1]
    
    # Global Average Pooling
    avg_pool = layers.GlobalAveragePooling2D()(inputs)
    max_pool = layers.GlobalMaxPooling2D()(inputs)
    
    # MLP
    mlp = layers.Dense(channels // ratio, activation='relu')
    
    avg_mlp = mlp(avg_pool)
    max_mlp = mlp(max_pool)
    
    mlp_output = layers.Add()([avg_mlp, max_mlp])
    
    attention = layers.Dense(channels, activation='softmax')(mlp_output)
    return layers.Reshape((1, 1, channels))(attention)

def spatial_attention_block(inputs):
    """
    Spatial Attention Module
    
    Parameters:
    - inputs: Input tensor
    """
    # Average pooling
    avg_pool = layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1, keepdims=True))(inputs)
    
    # Max pooling
    max_pool = layers.Lambda(lambda x: tf.reduce_max(x, axis=-1, keepdims=True))(inputs)
    
    # Concatenate
    concat = layers.Concatenate(axis=-1)([avg_pool, max_pool])
    
    # Convolution
    attention = layers.Conv2D(1, 7, padding='same', activation='softmax')(concat)
    
    return attention

# Create the model
model = mobile_unet_efficientnet(input_size=(256, 256, 3), num_classes=3)
model.summary()

Model: "256_MobileU-Net_EfficientNetB1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 256, 256, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_2 (Rescaling)       │ (None, 256, 256, 3)       │               0 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ normalization_1               │ (None, 256, 256, 3)       │               7 │ rescaling_2[0][0]          │
│ (Normalization)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_3 (Rescaling)       │ (None, 256, 256, 3)       │               0 │ normalization_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv_pad (ZeroPadding2D) │ (None, 257, 257, 3)       │               0 │ rescaling_3[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv (Conv2D)            │ (None, 128, 128, 32)      │             864 │ stem_conv_pad[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_bn (BatchNormalization)  │ (None, 128, 128, 32)      │             128 │ stem_conv[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_activation (Activation)  │ (None, 128, 128, 32)      │               0 │ stem_bn[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_dwconv                │ (None, 128, 128, 32)      │             288 │ stem_activation[0][0]      │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_bn                    │ (None, 128, 128, 32)      │             128 │ block1a_dwconv[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_activation            │ (None, 128, 128, 32)      │               0 │ block1a_bn[0][0]           │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_squeeze            │ (None, 32)                │               0 │ block1a_activation[0][0]   │
│ (GlobalAveragePooling2D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reshape (Reshape)  │ (None, 1, 1, 32)          │               0 │ block1a_se_squeeze[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reduce (Conv2D)    │ (None, 1, 1, 8)           │             264 │ block1a_se_reshape[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_expand (Conv2D)    │ (None, 1, 1, 32)          │             28

 Total params: 2,661,219 (10.15 MB)

 Trainable params: 2,521,142 (9.62 MB)

 Non-trainable params: 140,077 (547.18 KB)

# SegNet + EfficientNetB1

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB1

def create_segnet(input_shape=(256, 256, 3), num_classes=3):
    # Encoder (EfficientNetB1 as Backbone)
    backbone = EfficientNetB1(include_top=False, weights='imagenet', input_shape=input_shape)
    backbone.trainable = False  # Freeze backbone weights
    
    # Encoder Layers
    encoder_outputs = [
        backbone.get_layer(name).output 
        for name in ["block2a_expand_activation", "block3a_expand_activation", "block4a_expand_activation"]
    ]
    
    encoder = models.Model(inputs=backbone.input, outputs=encoder_outputs, name="EfficientNetB1_Encoder")
    
    # Decoder
    inputs = layers.Input(shape=input_shape)
    encoder_features = encoder(inputs)
    
    # Lightweight Decoder
    x = encoder_features[-1]  # Start with the deepest layer
    for features in reversed(encoder_features[:-1]):
        x = layers.Conv2DTranspose(filters=features.shape[-1], kernel_size=3, strides=2, padding="same")(x)
        x = layers.Concatenate()([x, features])
        x = layers.Conv2D(filters=features.shape[-1], kernel_size=3, activation='relu', padding='same')(x)
        x = layers.BatchNormalization()(x)
    
    # Final Segmentation Head
    x = layers.Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding="same")(x)
    x = layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Conv2D(num_classes, kernel_size=1, activation="softmax")(x)  # Multi-class output

    # SegNet Model
    model = models.Model(inputs, outputs, name="Lightweight_SegNet_EfficientNetB1")
    return model

# Create the model
segnet_model = create_segnet()
segnet_model.summary()

Model: "Lightweight_SegNet_EfficientNetB1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 128, 128, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ EfficientNetB1_Encoder        │ [(None, 64, 64, 96),      │         123,549 │ input_layer_1[0][0]        │
│ (Functional)                  │ (None, 32, 32, 144),      │                 │                            │
│                               │ (None, 16, 16, 240)]      │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_transpose              │ (None, 32, 32, 144)       │         311,184 │ EfficientNetB1_Encoder[0]… │
│ (Conv2DTranspose)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 32, 32, 288)       │               0 │ conv2d_transpose[0][0],    │
│                               │                           │                 │ EfficientNetB1_Encoder[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 32, 32, 144)       │         373,392 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 32, 32, 144)       │             576 │ conv2d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_transpose_1            │ (None, 64, 64, 96)        │         124,512 │ batch_normalization[0][0]  │
│ (Conv2DTranspose)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_1 (Concatenate)   │ (None, 64, 64, 192)       │               0 │ conv2d_transpose_1[0][0],  │
│                               │                           │                 │ EfficientNetB1_Encoder[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 64, 64, 96)        │         165,984 │ concatenate_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 64, 64, 96)        │             384 │ conv2d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_transpose_2            │ (None, 128, 128, 64)      │          55,360 │ batch_normalization_1[0][… │
│ (Conv2DTranspose)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 128, 128, 64)      │          36,928 │ conv2d_transpose_2[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 128, 128, 64)      │             25

 Total params: 1,192,320 (4.55 MB)

 Trainable params: 1,068,163 (4.07 MB)

 Non-trainable params: 124,157 (484.99 KB)

In [23]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetB1

def lightweight_segnet_fixed(input_shape=(256, 256, 3), num_classes=3):
    # Input Layer
    inputs = layers.Input(shape=input_shape)
    
    # EfficientNetB1 Backbone (pretrained, not include top)
    backbone = EfficientNetB1(include_top=False, input_shape=input_shape, weights='imagenet')
    x = backbone.output  # Output feature maps of size (4, 4, 1280)

    # Bottleneck
    x = layers.Conv2D(256, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Upsampling blocks
    x = layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(x)  # (8, 8, 128)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(x)  # (16, 16, 64)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same')(x)  # (32, 32, 32)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same')(x)  # (64, 64, 16)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(8, (3, 3), strides=(2, 2), padding='same')(x)  # (128, 128, 8)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Final Output Layer
    output = layers.Conv2D(num_classes, (1, 1), padding='same', activation='softmax')(x)  # (128, 128, num_classes)

    # Create the model
    model = Model(inputs, output)

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Build the corrected model
model_fixed = lightweight_segnet_fixed(input_shape=(256, 256, 3), num_classes=3)

# Summary of the corrected model
model_fixed.summary()

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_43 (InputLayer)   │ (None, 128, 128, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_44 (Rescaling)      │ (None, 128, 128, 3)       │               0 │ input_layer_43[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ normalization_22              │ (None, 128, 128, 3)       │               7 │ rescaling_44[0][0]         │
│ (Normalization)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_45 (Rescaling)      │ (None, 128, 128, 3)       │               0 │ normalization_22[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv_pad (ZeroPadding2D) │ (None, 129, 129, 3)       │               0 │ rescaling_45[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv (Conv2D)            │ (None, 64, 64, 32)        │             864 │ stem_conv_pad[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_bn (BatchNormalization)  │ (None, 64, 64, 32)        │             128 │ stem_conv[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_activation (Activation)  │ (None, 64, 64, 32)        │               0 │ stem_bn[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_dwconv                │ (None, 64, 64, 32)        │             288 │ stem_activation[0][0]      │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_bn                    │ (None, 64, 64, 32)        │             128 │ block1a_dwconv[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_activation            │ (None, 64, 64, 32)        │               0 │ block1a_bn[0][0]           │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_squeeze            │ (None, 32)                │               0 │ block1a_activation[0][0]   │
│ (GlobalAveragePooling2D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reshape (Reshape)  │ (None, 1, 1, 32)          │               0 │ block1a_se_squeeze[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reduce (Conv2D)    │ (None, 1, 1, 8)           │             264 │ block1a_se_reshape[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_expand (Conv2D)    │ (None, 1, 1, 32)          │             28

 Total params: 9,919,738 (37.84 MB)

 Trainable params: 9,856,675 (37.60 MB)

 Non-trainable params: 63,063 (246.34 KB)

In [24]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetB1

def lightweight_segnet_with_rescaling(input_shape=(256, 256, 3), num_classes=3):
    # Input Layer
    inputs = layers.Input(shape=input_shape, dtype='uint8')  # Mengatur dtype sebagai uint8

    # Lapisan Rescaling untuk normalisasi
    x = layers.Rescaling(1.0 / 255.0)(inputs)  # Normalisasi dari [0, 255] ke [0, 1]

    # EfficientNetB1 Backbone (pretrained, not include top)
    backbone = EfficientNetB1(include_top=False, input_tensor=x, weights='imagenet')
    x = backbone.output  # Output feature maps dari backbone

    # Bottleneck
    x = layers.Conv2D(256, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Upsampling blocks
    x = layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(8, (3, 3), strides=(2, 2), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Final Output Layer
    output = layers.Conv2D(num_classes, (1, 1), padding='same', activation='softmax')(x)

    # Create the model
    model = Model(inputs, output)

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Membuat model
model = lightweight_segnet_with_rescaling(input_shape=(256, 256, 3), num_classes=3)

# Menampilkan arsitektur model
model.summary()

Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_44 (InputLayer)   │ (None, 128, 128, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_46 (Rescaling)      │ (None, 128, 128, 3)       │               0 │ input_layer_44[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_47 (Rescaling)      │ (None, 128, 128, 3)       │               0 │ rescaling_46[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ normalization_23              │ (None, 128, 128, 3)       │               7 │ rescaling_47[0][0]         │
│ (Normalization)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_48 (Rescaling)      │ (None, 128, 128, 3)       │               0 │ normalization_23[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv_pad (ZeroPadding2D) │ (None, 129, 129, 3)       │               0 │ rescaling_48[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv (Conv2D)            │ (None, 64, 64, 32)        │             864 │ stem_conv_pad[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_bn (BatchNormalization)  │ (None, 64, 64, 32)        │             128 │ stem_conv[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_activation (Activation)  │ (None, 64, 64, 32)        │               0 │ stem_bn[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_dwconv                │ (None, 64, 64, 32)        │             288 │ stem_activation[0][0]      │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_bn                    │ (None, 64, 64, 32)        │             128 │ block1a_dwconv[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_activation            │ (None, 64, 64, 32)        │               0 │ block1a_bn[0][0]           │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_squeeze            │ (None, 32)                │               0 │ block1a_activation[0][0]   │
│ (GlobalAveragePooling2D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reshape (Reshape)  │ (None, 1, 1, 32)          │               0 │ block1a_se_squeeze[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reduce (Conv2D)    │ (None, 1, 1, 8)           │             26

 Total params: 9,919,738 (37.84 MB)

 Trainable params: 9,856,675 (37.60 MB)

 Non-trainable params: 63,063 (246.34 KB)

# DeepLabV3+ + EfficientNetB1

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def DeepLabV3P(input_shape=(256, 256, 3), num_classes=3, backbone="EfficientNetB1"):
    """DeepLabV3P with EfficientNetB1 Backbone"""
    # Input layer
    inputs = layers.Input(shape=input_shape)

    # Backbone (EfficientNetB1)
    base_model = tf.keras.applications.EfficientNetB1(
        include_top=False, weights="imagenet", input_tensor=inputs
    )

    # Extract intermediate feature maps
    high_level_features = base_model.get_layer("block6a_expand_activation").output  # Encoder
    low_level_features = base_model.get_layer("block2b_add").output  # Low-level skip

    # Low-level features processing (reduce channels)
    low_level_features = layers.Conv2D(48, (1, 1), padding="same", activation="relu")(low_level_features)
    low_level_features = layers.BatchNormalization()(low_level_features)

    # ASPP (Atrous Spatial Pyramid Pooling) Module
    aspp = layers.Conv2D(256, (1, 1), padding="same", activation="relu")(high_level_features)
    aspp = layers.BatchNormalization()(aspp)

    for rate in [6, 12, 18]:  # Atrous rates
        x = layers.Conv2D(256, (3, 3), padding="same", dilation_rate=rate, activation="relu")(high_level_features)
        x = layers.BatchNormalization()(x)
        aspp = layers.Concatenate()([aspp, x])

    # Image pooling
    image_pooling = layers.GlobalAveragePooling2D()(high_level_features)
    image_pooling = layers.Reshape((1, 1, -1))(image_pooling)
    image_pooling = layers.Conv2D(256, (1, 1), padding="same", activation="relu")(image_pooling)
    # Upsample to match ASPP feature map size
    image_pooling = layers.UpSampling2D(size=(aspp.shape[1] // image_pooling.shape[1], aspp.shape[2] // image_pooling.shape[2]))(image_pooling)

    # Concatenate ASPP outputs
    aspp = layers.Concatenate()([aspp, image_pooling])
    aspp = layers.Conv2D(256, (1, 1), padding="same", activation="relu")(aspp)
    aspp = layers.BatchNormalization()(aspp)

    # Decoder
    decoder = layers.UpSampling2D(size=(4, 4), interpolation="bilinear")(aspp)
    decoder = layers.Concatenate()([decoder, low_level_features])
    decoder = layers.Conv2D(256, (3, 3), padding="same", activation="relu")(decoder)
    decoder = layers.BatchNormalization()(decoder)
    decoder = layers.Conv2D(256, (3, 3), padding="same", activation="relu")(decoder)
    decoder = layers.BatchNormalization()(decoder)
    
    # Output layer
    decoder = layers.UpSampling2D(size=(4, 4), interpolation="bilinear")(decoder)  # Tambahkan upsampling tambahan
    outputs = layers.Conv2D(num_classes, (1, 1), activation="softmax")(decoder)

    # Define the model
    model = Model(inputs, outputs, name="DeepLabV3P_EfficientNetB1")

    return model


# Instantiate and compile the model
input_shape = (256, 256, 3)
num_classes = 3
model = DeepLabV3P(input_shape=input_shape, num_classes=num_classes)

# Summary of the model
model.summary()

Model: "DeepLabV3P_EfficientNetB1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 128, 128, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling (Rescaling)         │ (None, 128, 128, 3)       │               0 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ normalization (Normalization) │ (None, 128, 128, 3)       │               7 │ rescaling[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_1 (Rescaling)       │ (None, 128, 128, 3)       │               0 │ normalization[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv_pad (ZeroPadding2D) │ (None, 129, 129, 3)       │               0 │ rescaling_1[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv (Conv2D)            │ (None, 64, 64, 32)        │             864 │ stem_conv_pad[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_bn (BatchNormalization)  │ (None, 64, 64, 32)        │             128 │ stem_conv[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_activation (Activation)  │ (None, 64, 64, 32)        │               0 │ stem_bn[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_dwconv                │ (None, 64, 64, 32)        │             288 │ stem_activation[0][0]      │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_bn                    │ (None, 64, 64, 32)        │             128 │ block1a_dwconv[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_activation            │ (None, 64, 64, 32)        │               0 │ block1a_bn[0][0]           │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_squeeze            │ (None, 32)                │               0 │ block1a_activation[0][0]   │
│ (GlobalAveragePooling2D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reshape (Reshape)  │ (None, 1, 1, 32)          │               0 │ block1a_se_squeeze[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reduce (Conv2D)    │ (None, 1, 1, 8)           │             264 │ block1a_se_reshape[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_expand (Conv2D)    │ (None, 1, 1, 32)          │             288 │ block1a_se_reduce[0][0]    │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 7,924,958 (30.23 MB)

 Trainable params: 7,897,047 (30.12 MB)

 Non-trainable params: 27,911 (109.03 KB)

In [3]:
from tensorflow.keras.applications import EfficientNetB1

def DeepLabV3Plus(shape, num_classes):
    """ inputs """
    inputs = Input(shape)

    """ Pre-Trained EfficientNetB1 """
    base_model = EfficientNetB1(weights='imagenet', include_top=False, input_tensor=inputs)

    """ Pre-Trained EfficientNetB1 Output """
    image_features = base_model.get_layer('block6a_expand_activation').output  # High-level features
    x_a = ASSP(image_features)
    x_a = UpSampling2D((4, 4), interpolation="bilinear")(x_a)

    x_b = base_model.get_layer('block2b_add').output  # Low-level features
    x_b = Conv2D(filters=48, kernel_size=1, padding='same', use_bias=False)(x_b)
    x_b = BatchNormalization()(x_b)
    x_b = Activation("relu")(x_b)

    # Decoder: Combine high-level and low-level features
    x = Concatenate()([x_a, x_b])
    x = Conv2D(filters=256, kernel_size=3, padding='same', activation="relu", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(filters=256, kernel_size=3, padding='same', activation="relu", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = UpSampling2D((4, 4), interpolation="bilinear")(x)

    """ Outputs """
    x = Conv2D(num_classes, (1, 1), name='output_layer')(x)
    x = Activation('sigmoid')(x)  # Use 'softmax' for multi-class segmentation

    """ Model """
    model = Model(inputs=inputs, outputs=x)
    return model

# Instantiate and compile the model
input_shape = (256, 256, 3)
num_classes = 3
model = DeepLabV3P(input_shape=input_shape, num_classes=num_classes)

# Summary of the model
model.summary()

Model: "DeepLabV3P_EfficientNetB1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)    │ (None, 128, 128, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_4 (Rescaling)       │ (None, 128, 128, 3)       │               0 │ input_layer_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ normalization_2               │ (None, 128, 128, 3)       │               7 │ rescaling_4[0][0]          │
│ (Normalization)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_5 (Rescaling)       │ (None, 128, 128, 3)       │               0 │ normalization_2[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv_pad (ZeroPadding2D) │ (None, 129, 129, 3)       │               0 │ rescaling_5[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv (Conv2D)            │ (None, 64, 64, 32)        │             864 │ stem_conv_pad[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_bn (BatchNormalization)  │ (None, 64, 64, 32)        │             128 │ stem_conv[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_activation (Activation)  │ (None, 64, 64, 32)        │               0 │ stem_bn[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_dwconv                │ (None, 64, 64, 32)        │             288 │ stem_activation[0][0]      │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_bn                    │ (None, 64, 64, 32)        │             128 │ block1a_dwconv[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_activation            │ (None, 64, 64, 32)        │               0 │ block1a_bn[0][0]           │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_squeeze            │ (None, 32)                │               0 │ block1a_activation[0][0]   │
│ (GlobalAveragePooling2D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reshape (Reshape)  │ (None, 1, 1, 32)          │               0 │ block1a_se_squeeze[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reduce (Conv2D)    │ (None, 1, 1, 8)           │             264 │ block1a_se_reshape[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_expand (Conv2D)    │ (None, 1, 1, 32)          │             28

 Total params: 7,924,958 (30.23 MB)

 Trainable params: 7,897,047 (30.12 MB)

 Non-trainable params: 27,911 (109.03 KB)